In [48]:
# This is the template for the submission. If you want, you can develop your algorithm in a regular Python script and copy the code here for submission.

# Team members (e-mail, legi):
# zhisun@ethz.ch, 22-958-227
# enjcao@ethz.ch, 22-942-700
# yifzhou@ethz.ch, 22-940-381

In [69]:
import pandas as pd
import math
from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset
from os import listdir
from os.path import isfile, join
from math import sqrt
import numpy as np
from numpy.fft import fft
from scipy import signal
from scipy.fft import fft2
import matplotlib.pyplot as plt


In [7]:
# Get the path of all traces
dir_traces = 'data/recorded'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f)) and f!='.DS_Store']
filenames.sort()
# print(filenames[8])

In [14]:
# This function aims to find the component caused by gravity from data, which means the signal around 0 Hz
def get_gravity(data):
    filtered_data = np.zeros_like(data)
    # Parameters in IIR filter
    alpla = [1, -1.979133761292768, 0.979521463540373]
    beta = [0.000086384997973502, 0.00012769995947004, 0.000086384997973502]
    # Formula of IIR filter
    for i in range(2, len(data)):
        filtered_data[i] = alpla[0] * (data[i] * beta[0] + data[i-1] * beta[1] + data[i-2] * beta[2] - filtered_data[i-1] * alpla[1] - filtered_data[i-2] * alpla[2])
    return filtered_data

In [77]:
ids = []
location = []
ax_avg = []
ay_avg = []
az_avg = []

# for i, filename in enumerate(filenames):
filename = filenames[0]


    # Your algorithm goes here
    # Make sure, you only use data from the LilyGo Wristband, namely the following 10 keys (as in trace.data[key]):
    # 3-axis accelerometer: key in [ax, ay, az]
    # 3-axis gyro: key in [gx, gy, gz]
    # 3-axis magnetometer: key in [mx, my, mz]
    # IMU temperature: key==temperature


    # Get the original data from Lilygo -----------------------------------------------------------------------
    # Get accelerator data
trace = Recording(filename, no_labels=False, mute=True)
    

ax_original = trace.data['ax'].values
ay_original = trace.data['ay'].values
az_original = trace.data['az'].values

# Get time stamp and sampling rate
timestamp_raw = trace.data['ax'].timestamps
timestamp_ms = timestamp_raw*1000
sampling_rate = trace.data['ax'].samplerate



# divide feature windows
std_win = 3 #s
window_len = round(std_win*sampling_rate)

# get 2n sample windows, with n samples overlapping
windowed_data = [ax_original[s:s+window_len] for s in range(0, len(ax_original)-window_len, round(window_len/2))]
print("sampling rate: ", sampling_rate, ", total time: ", timestamp_ms[-1],", number of windows: ", len(windowed_data))
# get features from sliding windows
windowed_dc = np.mean(windowed_data, axis=1)
windowed_energy = np.sum(np.square(fft(windowed_data, axis=1)),axis=1)/window_len





# ids.append(filename)
# location.append(trace.labels['board_loc'])
# ax_avg.append(np.average(ax_abs))
# ay_avg.append(np.average(ay_abs))
# az_avg.append(np.average(az_abs))
# print("location: ", trace.labels['board_loc'], "ax: ", np.average(ax_abs), "ay: ", np.average(ay_abs), "az: ", np.average(az_abs))


# Visualize the original signal, filtered signal and detected peaks -----------------------------------------
# title = "location: "+str(trace.labels['board_loc'])+" ax: "+str(np.average(ax_abs))+" ay: "+str(np.average(ay_abs))+" az: "+str(np.average(az_abs))
# fig, axes1 = plt.subplots(3,1, figsize=(60, 5)) #figsize is width, height
# axes1[0].set_title(title)
# axes1[0].plot(ax_abs, alpha=1, label="ax mag")
# axes1[1].plot(ay_abs, alpha=1, label="ay mag")
# axes1[2].plot(az_abs, alpha=1, label="az mag")



sampling rate:  199.76613651042464 , total time:  710671.0 , number of windows:  472


In [28]:
# Write the detected step counts into a .csv file to then upload the .csv file to Kaggle
# When cross-checking the .csv file on your computer, we recommend using the text editor and NOT excel so that the results are displayed correctly
# IMPORTANT: Do NOT change the name of the columns ('Id' and 'Predicted') of the .csv file
df = pd.DataFrame({'Id':ids,'Location':location,'ax':ax_avg,'ay':ay_avg,'az':az_avg})
df.to_csv('./location.csv', index=False)